In [121]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

Hacer lista con los equipos por pais

In [ ]:
LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]

In [122]:
url = "https://www.promiedos.com.ar/torneo=1045"
r = rq.get(url)
sopa = bs(r.content, "html.parser")
lista_fases = sopa.find_all('div', id='partoff')
lista_grupos = sopa.find_all('div', class_ = 'grupo')

In [123]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
resultados = []
goles = []
dif_goles = []
dias = []
anos = []
fases = []

In [124]:
final_unica = sopa.find_all('div', id='partoff1')
if final_unica != []:
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ1 = equ[1].text ; equ2 = equ[2].text
    locales.append(equ1)
    visitas.append(equ2)

    res1 = final_unica[0].find_all('td', class_='game-r1')
    res1 = (res1[1].text)[0]
    gol_locales.append(int(res1))
    res2 = final_unica[0].find_all('td', class_='game-r2')
    res2 = (res2[1].text)[0]
    gol_visitas.append(int(res2))

    if int(res1) == int(res2):
        resultados.append("Empate")
        dif_goles.append(0)

    elif int(res1) > int(res2):
        resultados.append("Local")
        dif_goles.append(int(res1) - int(res2))

    elif int(res1) < int(res2):
        resultados.append("Visita")
        dif_goles.append(int(res2) - int(res1))

    goles.append(int(res1) + int(res2))


    dia = final_unica[0].find_all('tr', class_='diapart')
    dia = dia[0].text
    dia = dia.split(" ")
    anos.append(dia[4])
    ano = dia[4]
    dia = dia[1] + " " + dia[2] + " " + dia[3] + " " + dia[4]
    dias.append(dia)
        

In [129]:
cont = 0
for data in lista_fases:
    lista_partidos = data.find_all('div', id='fixturein')
    fase = LISTA_RONDAS[cont]
    for datos in lista_partidos:
        equ = datos.find_all('td', class_='game-t1')
        equ1 = equ[1].text ; equ2 = equ[2].text
        locales.append(equ1)
        visitas.append(equ2)

        res1 = datos.find_all('td', class_='game-r1')
        res1 = (res1[1].text)[0]
        gol_locales.append(int(res1))
        res2 = datos.find_all('td', class_='game-r2')
        res2 = (res2[1].text)[0]
        gol_visitas.append(int(res2))

        if int(res1) == int(res2):
            resultados.append("Empate")
            dif_goles.append(0)

        elif int(res1) > int(res2):
            resultados.append("Local")
            dif_goles.append(int(res1) - int(res2))

        elif int(res1) < int(res2):
            resultados.append("Visita")
            dif_goles.append(int(res2) - int(res1))

        goles.append(int(res1) + int(res2))


        dia = datos.find_all('tr', class_='diapart')
        dia = dia[0].text
        dia = dia.split(" ")
        anos.append(dia[4])
        ano = dia[4]
        dia = dia[1] + " " + dia[2] + " " + dia[3] + " " + dia[4]
        dias.append(dia)

        fases.append(fase)

    cont += 1
    print(cont)
    print(fase)
        
    

1
Final
2
Final
3
Semifinal
4
Semifinal
5
Semifinal
6
Semifinal
7
Cuartos
8
Cuartos


In [126]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores["Visita"] = visitas
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias

In [127]:
df_libertadores

,Local,Visita,Gol_local,Gol_visita,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Gremio,Lanus,1,0,Local,2017,Final,1,1,22 de Noviembre 2017
1,Lanus,Gremio,1,2,Visita,2017,Final,3,1,29 de Noviembre 2017
2,River Plate,Lanus,1,0,Local,2017,Semifinal,1,1,24 de Octubre 2017
3,Lanus,River Plate,4,2,Local,2017,Semifinal,6,2,31 de Octubre 2017
4,Barcelona (E),Gremio,0,3,Visita,2017,Semifinal,3,3,25 de Octubre 2017
5,Gremio,Barcelona (E),0,1,Visita,2017,Semifinal,1,1,01 de Noviembre 2017
6,Wilstermann,River Plate,3,0,Local,2017,Semifinal,3,3,14 de Septiembre 2017
7,River Plate,Wilstermann,8,0,Local,2017,Semifinal,8,8,21 de Septiembre 2017
8,San Lorenzo,Lanus,2,0,Local,2017,Semifinal,2,2,13 de Septiembre 2017
9,Lanus,San Lorenzo,2,0,Local,2017,Semifinal,2,2,21 de Septiembre 2017
